## 광고 시뮬레이션 모형 구축

### 1. 데이터 불러오기 및 전처리
---

In [ ]:
import pandas as pd
import numpy as np
import pymysql
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.special import boxcox, inv_boxcox
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import gc
import os
import csv

# 그래프 관련 패키지
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font', family = 'NANUMBARUNGOTHIC')
print(plt.rcParams['font.family'])

%matplotlib inline

['NANUMBARUNGOTHIC']


In [ ]:
# DB 접속(정보 생략)
conn = pymysql.connect()

In [ ]:
# 네이버 데이터 쿼리 호출
query = 'SELECT statDate, t1.campaignId, adGroupId, adGroupName, adGroupTypeName, deviceType, \
                SUM(amtExecution) AS amtExecution, SUM(adCost) AS adCost, SUM(impressionCnt) AS impressionCnt, SUM(clickCnt) AS clickCnt, SUM(viewCnt) AS viewCnt, \
                SUM(firstQuarterPlayCnt) AS firstQuarterPlayCnt, SUM(midpointPlayCnt) AS midpointPlayCnt, SUM(thirdQuarterPlayCnt) AS thirdQuarterPlayCnt, SUM(completePlayCnt) AS completePlayCnt, \
                advertiser, brand, businessMain, businessSub, NULL AS userConvCnt\
        FROM naverNospAdStatDaily AS t1, naverNospCampaign AS t2 WHERE t1.campaignId = t2.campaignId AND t1.statDate BETWEEN "20190101" and "20190930"\
        GROUP BY statDate, t1.campaignId, adGroupId, adGroupName, adGroupTypeName, deviceType\
        UNION\
        SELECT statDate, t3.campaignId, adGroupId, adGroupName, adGroupTypeName, deviceType, \
                SUM(amtExecution) AS amtExecution, SUM(adCost) AS adCost, SUM(impressionCnt) AS impressionCnt, SUM(clickCnt) AS clickCnt, SUM(viewCnt) AS viewCnt, \
                SUM(firstQuarterPlayCnt) AS firstQuarterPlayCnt, SUM(midpointPlayCnt) AS midpointPlayCnt, SUM(thirdQuarterPlayCnt) AS thirdQuarterPlayCnt, SUM(completePlayCnt) AS completePlayCnt,\
                advertiser, brand, businessMain, businessSub, userConvCnt\
        FROM CPMSDB.naverNospAdStatDaily AS t3, CPMSDB.naverNospCampaign AS t4 WHERE t3.campaignId = t4.campaignId AND t3.statDate >= "20191001"\
        GROUP BY statDate, t3.campaignId, adGroupId, adGroupName, adGroupTypeName, deviceType'

# query1 = 'SELECT DISTINCT t1.campaignId, t3.brandId AS cmBrandId, t3.brandName AS cmBrandName, t4.companyId AS cmCompanyId, t4.companyName AS cmCompanyName,\
#                 t5.businessTypeId AS cmBusinessTypeId, t5.businessTypeName AS cmBusinessTypeName, t6.categoryId AS cmCategoryId, t6.categoryName AS cmCategoryName\
#         FROM CPMSDB.CpmsMediaDataLink AS t1, CPMSDB.CpmsCampaign AS t2, rainbow.Brand AS t3, rainbow.Company AS t4, rainbow.BusinessType AS t5, rainbow.BusinessCategory AS t6\
#         WHERE t1.cpmsCampaignId = t2.campaignId AND t2.brandId = t3.brandId AND t3.companyId = t4.companyId AND t5.businessTypeId = t6.businessTypeId AND t3.categoryId = t6.categoryId\
#                 AND t1.mediaCode = "008001" AND t2.deleteFlag = "N"'### name 정보 추가!

df = pd.read_sql(query, conn) # CPMSDB와 datacenterdb의 데이터 추출
# brand_cpms = pd.read_sql(query1, conn) # 기입력된 CPMSDB의 광고주 정보를 가져옴.
# brand_cpms['campaignId'] = brand_cpms['campaignId'].astype('int64') # campaignId가 'string' 형태로 들어오기 때문에 정수형으로 변경

# df = pd.merge(df, brand_cpms, how = 'left', on = 'campaignId') # CPMSDB의 광고주 정보 붙이기

In [ ]:
df = df[df.statDate < '20210501'] ###
df.head()

,statDate,campaignId,adGroupId,adGroupName,adGroupTypeName,deviceType,amtExecution,adCost,impressionCnt,clickCnt,viewCnt,firstQuarterPlayCnt,midpointPlayCnt,thirdQuarterPlayCnt,completePlayCnt,advertiser,brand,businessMain,businessSub,userConvCnt
0,20190101,1217984,1930902,M_메인_연애결혼_브랜딩DA,배너_이미지형,M,571440.0,571440.0,253094.0,1254.0,0.0,0.0,0.0,0.0,0.0,엘오케이(유),비오템,화장품/생활용품,화장품,NaN
1,20190101,1218616,1950642,P_사전_우측배너_통합,배너_이미지/플래시형,P,410589.0,373220.0,471928.0,213.0,0.0,0.0,0.0,0.0,0.0,사이버한국외국어대학교,사이버한국외국어대학교,교육/출판,사이버대학교,NaN
2,20190101,1218943,1935181,M_메인_여행_콘텐츠DA_가로플리킹형,콘텐츠_메인_가로플리킹형,M,714288.0,714288.0,219994.0,5031.0,0.0,0.0,0.0,0.0,0.0,주식회사 노랑풍선,노랑풍선,서비스,여행/숙박,NaN
3,20190101,1218983,1935360,M_메인_여행_콘텐츠DA_가로플리킹형,콘텐츠_메인_가로플리킹형,M,714288.0,714288.0,219748.0,3357.0,0.0,0.0,0.0,0.0,0.0,"캐나다 관광청, 한국사무소",캐나다관광청,관공서/단체,단체,NaN
4,20190101,1218988,1937350,PM_네이버 동영상_15초 SKIP,동영상_15초 SKIP형,M,955860.0,796550.0,56896.0,339.0,51058.0,54405.0,52471.0,51704.0,51002.0,엘오케이(유),키엘,화장품/생활용품,화장품,NaN


In [ ]:
df.shape # 27,764*20

(27764, 20)

In [ ]:
# 브랜드 및 상품 매핑 자료 업로드
# 매월 업데이트 필요
query_brand = 'SELECT brandName AS brand, advertiserName AS advertiser, businessMain, businessSub,\
            cmBrandName, cmCompanyName, cmBusinessTypeName, cmCategoryName, cmBrandId, cmCompanyId, cmBusinessTypeId, cmCategoryId\
            FROM naverNospBrand'
query_slot = 'SELECT * FROM naverNospSlot'

nosp_brand = pd.read_sql(query_brand, conn)
nosp_slot = pd.read_sql(query_slot, conn)

In [ ]:
# 업데이트 시 브랜드가 없을 경우 csv로 저장 후 브랜드 매핑 작업 후 다시 진행
# df2 = df[df.cmBrandId.isnull()]
# df2 = df2[['adGroupId','advertiser','brand','businessMain', 'businessSub']]
# print(len(df2.drop_duplicates())) # CPMS에 등록되지 않은 브랜드 수: 2,817 ###

df2 = pd.merge(df, nosp_brand, how = 'left', on = ['brand', 'advertiser','businessMain', 'businessSub']) ###

brandNA = df2[df2.cmBusinessTypeId.isnull()]
brandNA = brandNA.drop(['adGroupId'], axis = 1)
brandNA = brandNA[['brand','advertiser','businessMain','businessSub','cmBrandId','cmCompanyId','cmBusinessTypeId','cmCategoryId',
                   'cmBrandName', 'cmCompanyName','cmBusinessTypeName','cmCategoryName']] ###
brandNA = brandNA.drop_duplicates()

print(len(brandNA)) # 50
brandNA
#brandNA.to_csv('/home/anaconda3/da/data/brandNA_naver.csv')

50


In [ ]:
# 캠페인 기간을 구하기 위하여 캠페인 시작일, 종료일을 각각 계산
start = pd.DataFrame(df['statDate'].groupby(df['adGroupId']).min())
end = pd.DataFrame(df['statDate'].groupby(df['adGroupId']).max())
print(start.head())
print(end.head())

In [ ]:
# 캠페인 상품별(adGroupId 기준) 수치 합계 계산
df1 = df[['adGroupId','amtExecution','adCost','impressionCnt','clickCnt', 'firstQuarterPlayCnt', 'midpointPlayCnt', 'thirdQuarterPlayCnt', 'completePlayCnt']]
df_summary = df1.groupby(df1['adGroupId']).sum()
df_summary.head()

In [ ]:
# 광고주 CM리포트 기준으로 맞추고 테이블 생성
# CPMSDB에 광고주 정보가 등록된 경우는 따로 매핑하지 않아도 되므로 brandId가 없는 캠페인만 등록
# CPMSDB는 ID값들만 등록하고 명칭은 따로 등록하지 않음
df2 = df2[['adGroupId','cmCompanyName','cmBrandName','cmBusinessTypeName', 'cmCategoryName', 'cmBrandId', 'cmCompanyId', 'cmBusinessTypeId', 'cmCategoryId']]
brand = df2.groupby(['adGroupId']).first()
# brand2 = df[['adGroupId', 'cmCompanyName','cmBrandName','cmBusinessTypeName', 'cmCategoryName',
#              'cmBrandId', 'cmCompanyId', 'cmBusinessTypeId', 'cmCategoryId']].dropna(subset=['cmBrandId']) # 브랜드 정보가 있는 CPMSDB의 데이터 추출 ###
# brand2 = brand2.groupby(['adGroupId']).first()

# brand = pd.concat([brand, brand2], join = 'outer') # 광고주 정보 이름이 없는 경우도 있음

brand.head()

In [ ]:
nosp_slot.head()

In [ ]:
# 정의되지 않은 상품명 저장
df3 = df[['adGroupId','adGroupName', 'adGroupTypeName']]
df3 = pd.merge(df3, nosp_slot, how = 'left', on = ['adGroupName', 'adGroupTypeName'])
slotNA = df3[df3.cmSlotId.isnull()]
slotNA = slotNA.drop(['adGroupId'], axis = 1)
slotNA = slotNA.drop_duplicates()
print(len(slotNA)) # 3
slotNA
#slotNA.to_csv('/home/anaconda3/da/data/slotNA_naver.csv')

In [ ]:
df3[df3.cmSlotName.isnull()].adGroupName.unique()
# 아래 상품 리스트는 테스트 성이거나 데이터가 많지 않아 분석에 활용하지 않음.
# M_신제품검색, PM_네이버 동영상_5초 SKIP(비보장), P_서칭뷰(신)

In [ ]:
# 상품명 완벽 통일 이후 아래 코드 수행
df3 = df3[['cmSlotName', 'cmBillingType', 'cmSlotId']]
slot = df3.groupby(df['adGroupId']).first()
slot.head()

In [ ]:
# 브랜드, 상품명, 캠페인 시작, 종료일을 포함한 분석 테이블 생성
df_temp = pd.merge(df_summary, start, how = 'left', left_index = True, right_index = True)
df_temp = pd.merge(df_temp, end, how = 'left', left_index = True, right_index = True)
df_temp = pd.merge(df_temp, brand, how = 'left', left_index = True, right_index = True)
df_temp = pd.merge(df_temp, slot, how = 'left', left_index = True, right_index = True)

print(len(df_temp)) # 3,324
df_temp.head()

In [ ]:
# 필요없는 데이터프레임을 삭제하고 메모리 최적화
del df1
del df2
del df3
del df_summary
del start
del end
del brand
del slot
del nosp_slot
del nosp_brand

gc.collect()

In [ ]:
# 캠페인 시작일, 종료일의 형식을 바꾸고 기간 계산
# 단, 시작일과 종료일이 같은 경우 0으로 나타나므로 +1을 하여 계산
df_temp = df_temp.rename(columns = {'statDate_x': 'start', 'statDate_y': 'end'})
df_temp['start'] = pd.to_datetime(df_temp['start'], format='%Y%m%d').dt.strftime("%Y-%m-%d").astype('datetime64[ns]')
df_temp['end'] = pd.to_datetime(df_temp['end'], format='%Y%m%d').dt.strftime("%Y-%m-%d").astype('datetime64[ns]')

df_temp['duration'] = df_temp['end'] - df_temp['start']
df_temp['duration'] = df_temp['duration'].astype('timedelta64[D]') + 1 # 캠페인 시작일과 종료일이 같은 경우는 기간이 0 이므로 +1

df_temp['startMonth'] = df_temp['start'].dt.month
df_temp['startweekday'] = df_temp['start'].dt.weekday

df_temp.head()

In [ ]:
# 결측값 확인
print(df_temp.isnull().sum())
print(df_temp[df_temp['impressionCnt'] == 0].count()) # 광고비 기준에서 노출로 변경(광고비는 0이지만 노출 가능!)

In [ ]:
# # 카테고리1 & 슬롯 ID 없는 데이터 제거 (브랜드, 광고주, 카테고리2까지는 없어도 가능하나...최소한 카테고리1 데이터는 있어야 함)
# df_temp = df_temp.dropna(subset = ['cmBusinessTypeId'])
# df_temp = df_temp.dropna(subset = ['cmSlotId'])
# print(len(df_temp)) # 3,290
# print(df_temp.isnull().sum())

In [ ]:
# 상품 유형별 구분
# 상품명 최종으로 매핑 후 아래 코드로 CM상품명으로 코드 수행
df_video = df_temp[df_temp.cmSlotName.str.contains('동영상') == True]
df_da = df_temp[df_temp.cmSlotName.str.contains('동영상') == False]
df_da_cpm = df_da[df_da.cmBillingType == 'cpm']
df_da_cpt = df_da[df_da.cmBillingType == 'cpt']
df_da_cpm.head()

In [ ]:
# 상품별 광고비 비교: 롤링보드 > 브랜딩DA > 화보DA > M 통합 DA > 스마트채널(스마트채널 순위 상승)
slot_cost = df_da_cpm['adCost'].groupby(df_da_cpm['cmSlotName']).sum()
slot_cost = pd.DataFrame(slot_cost)
slot_cost['total'] = slot_cost.adCost.sum()

slot_cost['costRatio'] = slot_cost.adCost / slot_cost.total * 100
slot_cost.sort_values(by = 'adCost', ascending = False)

In [ ]:
# 상품별 캠페인 횟수 비교: 롤링보드 > 브랜딩DA > M 밴드 띠배너 > M 통합 DA > M 웹툰 빅배너
slot_count = df_da_cpm['adCost'].groupby(df_da_cpm['cmSlotName']).count()
slot_count = pd.DataFrame(slot_count)
slot_count['total'] = slot_count.adCost.sum()

slot_count['costRatio'] = slot_count.adCost / slot_count.total * 100
slot_count.sort_values(by = 'adCost', ascending = False)

In [ ]:
# 업종별 광고비 비교: 건설, 건재 및 부동산 > 화장품 및 보건용품 > 패션/의류
category1_cost = df_da_cpm['adCost'].groupby(df_da_cpm['cmBusinessTypeName']).sum()
category1_cost = pd.DataFrame(category1_cost)
category1_cost['total'] = category1_cost.adCost.sum()

category1_cost['costRatio'] = category1_cost.adCost / category1_cost.total * 100
category1_cost.sort_values(by = 'adCost', ascending = False)

In [ ]:
#  광고주별 광고비 비교: 엘오케이 > 리치몬트코리아 > 부영주택 > SK건설 > 제일건설
advertiser_cost = df_da_cpm['adCost'].groupby(df_da_cpm['cmCompanyName']).sum()
advertiser_cost = pd.DataFrame(advertiser_cost)
advertiser_cost['total'] = advertiser_cost.adCost.sum()

advertiser_cost['costRatio'] = advertiser_cost.adCost / advertiser_cost.total * 100
advertiser_cost.sort_values(by = 'adCost', ascending = False)

In [ ]:
#  브랜드별 광고비 비교: SK뷰 > 창원월영마린애시앙 > 입생로랑 > 제일풍경채 > 반클리프아펠
brand_cost = df_da_cpm['adCost'].groupby(df_da_cpm['cmBrandName']).sum()
brand_cost = pd.DataFrame(brand_cost)
brand_cost['total'] = brand_cost.adCost.sum()

brand_cost['costRatio'] = brand_cost.adCost / brand_cost.total * 100
brand_cost.sort_values(by = 'adCost', ascending = False)

In [ ]:
# 광고주, 브랜드, 카테고리1, 카테고리2 별로 캠페인 횟수 막대그래프로 확인
category_feature = ['cmCompanyName', 'cmBrandName', 'cmBusinessTypeName', 'cmCategoryName']

for col in category_feature:
    plt.figure(figsize = (20,5))
    df_da_cpm[col].value_counts().plot(kind = 'bar')
    plt.title(col)
    plt.show()

In [ ]:
# 수치형 변수들의 분포 확인: 대부분의 캠페인 기간이 일주일 단위로 이루어짐
numerical_feature = ['amtExecution', 'adCost', 'impressionCnt', 'clickCnt', 'duration', 'startMonth', 'startweekday']

for col in numerical_feature:
    sns.distplot(df_da_cpm.loc[df_da_cpm[col].notnull(), col], kde_kws = {"bw": 1.5})
    plt.title(col)
    plt.show()

In [ ]:
for col in numerical_feature:
    plt.figure(figsize=(18,6))
    sns.boxplot(x = 'cmBusinessTypeName', y = col, data = df_da_cpm)
    plt.title('CM_카테고리1 - {}'.format(col))
    plt.xlabel('CM_카테고리1')
    plt.show()

In [ ]:
# 수치들의 왜도, 첨도 확인
print("Skewness: %f" % df_temp['amtExecution'].skew())
print("Kurtosis: %f" % df_temp['amtExecution'].kurt())

In [ ]:
sns.pairplot(x_vars=["clickCnt"], y_vars=["amtExecution"], data = df_da_cpm, hue="cmSlotName", size=5)

In [ ]:
# 변수들간의 상관관계 그래프로 확인
corrmat = df_da_cpm.corr()
f, ax = plt.subplots(figsize = (12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

### 3. 모형 구축
---

#### 1) LightGBM 모델: 직접 CTR을 label로 잡아서 추정하는 모델

In [ ]:
# 분석에 필요한 변수만 추출
df_da_cpm = df_da_cpm[['startMonth', 'startweekday', 'duration', 'cmBrandId', 'cmBrandName', 
                       'cmCompanyId', 'cmCompanyName', 'cmBusinessTypeId', 'cmBusinessTypeName', 'cmCategoryId', 
                       'cmCategoryName', 'cmSlotId', 'cmSlotName', 'amtExecution', 'adCost', 'impressionCnt', 'clickCnt']]

# CTR 계산
df_da_cpm['ctr'] = df_da_cpm.clickCnt / df_da_cpm.impressionCnt * 100
df_da_cpm.head()

In [ ]:
# 분석하고자 하는 변수 선택
label_ctr = 'ctr'
feature_ctr = ['amtExecution', 'startMonth', 'startweekday', 'cmBrandId', 'cmCompanyId','cmBusinessTypeId', 'cmCategoryId', 'cmSlotId']

X_ctr = df_da_cpm[feature_ctr]
y_ctr = df_da_cpm[label_ctr]

In [ ]:
# 변수의 왜도와 첨도 모두 높을 경우 변환 처리
X_ctr = X_ctr + 1 # boxcox 변환은 0보다 큰 값만 변환 가능
X_ctr_amtExecution, fitted_lambda = stats.boxcox(X_ctr.amtExecution)
np.save('/home/anaconda3/da/data/nosp_cpm/fitted_lambda_cpm', fitted_lambda) # 람다값 저장(CPMS에서 입력되는 값은 원래값 변환을 위해 저장)

boxcox = pd.DataFrame(X_ctr_amtExecution)
boxcox_table = pd.concat([X_ctr.amtExecution, boxcox], axis = 1)
X_ctr.amtExecution = X_ctr_amtExecution
print(X_ctr.head())

In [ ]:
# boxcox 변환은 0보다 큰 값만 변환할 수 있으므로 1을 더해주어 분석하고 추정값을 역변환 후 1을 빼준다.
y_ctr = y_ctr + 1
boxcox_y_ctr = stats.boxcox(y_ctr, fitted_lambda)
y_ctr = pd.DataFrame(boxcox_y_ctr)

In [ ]:
# 트레이닝 / 테스트 셋으로 데이터 분리
from sklearn.model_selection import train_test_split

X_train_ctr, X_test_ctr, y_train_ctr, y_test_ctr = train_test_split(X_ctr, y_ctr, test_size=0.3, random_state=42)

test_set_1 = X_test_ctr.copy()
test_set_1['ctr'] = y_test_ctr

print(X_train_ctr.shape, y_train_ctr.shape) # 1,085
print(X_test_ctr.shape, y_test_ctr.shape) # 465

In [ ]:
# GBM 모델을 수행하기 위한 데이터 셋 구성
train_ctr_ds = lgb.Dataset(X_train_ctr, label = y_train_ctr) 
test_ctr_ds = lgb.Dataset(X_test_ctr, label = y_test_ctr)

In [ ]:
# 최적의 learning_rate 설정
np.random.seed(1) # seed 고정

num_epoch = 100

best_r2 = 0
best_epoch = None
best_learning_rate = None

for epoch in range(num_epoch):
    
    learning_rate = np.random.uniform(low = 0 , high = 1)
    
    params_ctr = {'n_estimators': 100,
              'learning_rate': learning_rate,
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'mse', 
              'is_training_metric': True, 
              'random_state':42}
    
    model_ctr = lgb.train(params_ctr, train_ctr_ds)

    predict_train_ctr = model_ctr.predict(X_train_ctr)
    predict_test_ctr = model_ctr.predict(X_test_ctr)

    # boxcox 역치환
    y_test_ctr_inv = inv_boxcox(y_test_ctr, fitted_lambda)
    y_test_ctr_inv = y_test_ctr_inv - 1

    predict_test_ctr_inv = inv_boxcox(predict_test_ctr, fitted_lambda)
    predict_test_ctr_inv = predict_test_ctr_inv - 1

    predict_test_ctr_inv = pd.DataFrame(predict_test_ctr_inv)
    predict_test_ctr_inv[predict_test_ctr_inv < 0].count()

    # 다양한 형태의 연속형 값의 'GBM'모델 예측 정확도 확인
#     mse_ctr = mean_squared_error(y_test_ctr_inv, predict_test_ctr_inv)
#     rmse_ctr = mean_squared_error(y_test_ctr_inv, predict_test_ctr_inv)**0.5
#     mape_ctr = abs(y_test_ctr_inv - predict_test_ctr_inv)/y_test_ctr_inv
#     mape_ctr = mape_ctr.mean()
    r2_ctr = r2_score(y_test_ctr_inv, predict_test_ctr_inv)
    
    if r2_ctr > best_r2:
        
        best_r2 = r2_ctr
        best_epoch = epoch
        best_learning_rate = learning_rate

        print("epoch = {0: 4}, n_estimators = 100, learning_rate = {1: .3f}, R2 = {2: .4f}".format(epoch, learning_rate, r2_ctr))

print("----" * 15)
print("epoch = {0: 4}, n_estimators = 100, learning_rate = {1: .3f}, R2 = {2: .4f}".format(best_epoch, best_learning_rate, best_r2))
# n_estimators =  100, learning_rate =  0.288, R2 =  0.8097

In [ ]:
# 최적의 n_estimotrs 설정
n_estimators = [100, 300, 500, 1000, 5000, 10000]

best_r2 = 0
best_epoch = None
best_n_estimators = None

for epoch in range(len(n_estimators)):
    
    learning_rate = 100 / n_estimators[epoch] * best_learning_rate
    
    params_ctr = {'n_estimators': n_estimators[epoch],
              'learning_rate': learning_rate,
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'mse', 
              'is_training_metric': True, 
              'random_state':42}
    
    model_ctr = lgb.train(params_ctr, train_ctr_ds)

    predict_train_ctr = model_ctr.predict(X_train_ctr)
    predict_test_ctr = model_ctr.predict(X_test_ctr)

    # boxcox 역치환
    y_test_ctr_inv = inv_boxcox(y_test_ctr, fitted_lambda)
    y_test_ctr_inv = y_test_ctr_inv - 1

    predict_test_ctr_inv = inv_boxcox(predict_test_ctr, fitted_lambda)
    predict_test_ctr_inv = predict_test_ctr_inv - 1

    predict_test_ctr_inv = pd.DataFrame(predict_test_ctr_inv)
    predict_test_ctr_inv[predict_test_ctr_inv < 0].count()

    # 다양한 형태의 연속형 값의 'GBM'모델 예측 정확도 확인
#     mse_ctr = mean_squared_error(y_test_ctr_inv, predict_test_ctr_inv)
#     rmse_ctr = mean_squared_error(y_test_ctr_inv, predict_test_ctr_inv)**0.5
#     mape_ctr = abs(y_test_ctr_inv - predict_test_ctr_inv)/y_test_ctr_inv
#     mape_ctr = mape_ctr.mean()
    r2_ctr = r2_score(y_test_ctr_inv, predict_test_ctr_inv)
    
    if r2_ctr > best_r2:
        
        best_r2 = r2_ctr
        best_epoch = epoch
        best_n_estimators = n_estimators[epoch]
        best_learning_rate2 = learning_rate
        
        print("epoch = {0: 4}, n_estimators = {1: 4}, learning_rate = {2: .3f}, R2 = {3: .4f}".format(epoch, n_estimators[epoch], learning_rate, r2_ctr))
        
print("----" * 15)
print("epoch = {0: 4}, n_estimators = {1: 4}, learning_rate = {2: .3f}, R2 = {3: .4f}".format(best_epoch, best_n_estimators, best_learning_rate2, best_r2))
# n_estimators =  100, learning_rate =  0.288, R2 =  0.8097(최종)

In [ ]:
# 하이퍼 파라미터 세팅 (업데이트 마다 최적값으로 변경)
# n_estimators, learning_rate 이외의 하이퍼파라미터는 큰 차이가 없어 제외하였음.
params_ctr = {'n_estimators': best_n_estimators,
              'learning_rate': best_learning_rate2,
              'boosting': 'gbdt', 
              'objective': 'regression', 
              'metric': 'mse', 
              'is_training_metric': True, 
              'random_state':42}

model_ctr = lgb.train(params_ctr, train_ctr_ds)

In [ ]:
# LightGBM Feature Importance: amtExecution > cmBrandId > startMonth
fig, ax = plt.subplots(figsize=(15, 7))
lgb.plot_importance(model_ctr, max_num_features=30, ax=ax)
plt.title("LightGBM - Feature Importance")

In [ ]:
# plt.scatter(y_test_ctr_inv, predict_test_ctr_inv)
# ax = sns.regplot(x = y_test_ctr_inv, y = predict_test_ctr_inv)
# ax.figure.set_size_inches(6, 6)

In [ ]:
from sklearn.externals import joblib

# joblib.dump(model_ctr, '/home/anaconda3/da/data/nosp_cpm/nosp_cpm_model.pkl') # 모델 저장